## Kubernetes

*Caveat: Unless we have a truly massive or complex system, we probably don’t need Kubernetes, and using it should be the result of a deliberate cost benefit analysis in comparison to other hosted solutions or managed solutions.*

### Introduction

 - [Kubernetes](https://kubernetes.io/) is an open-source system for automating deployment, scaling, and management of containerized applications.
 - It was open-sourced by Google. Its predecessor was called `borg` internally.
 - Kubernetes, or K8s for short, is a central orchestration system in large complex software systems.
 - It coordinates a collection or cluster of computers with the purpose of working as a single unit. 
 - When we orchestrate containers with it, we are agnostic to which computer runs which container.
 - K8s takes care of scheduling and distribution of containers to these *unnamed* computers behind the scenes.

 - We will primarily be interested in ML model deployment.

 - Lets start by going over some of the key concepts in the K8s container orchestration system below. For more information, have a look at the basics documented at [https://kubernetes.io/docs/tutorials/kubernetes-basics/](https://kubernetes.io/docs/tutorials/kubernetes-basics/) and go from there.

 - A K8s cluster has two resources:
  - Master: coordinator of the cluster
  - Nodes: workers are essentially VMs or computers that run containers.

	![k8s1](images/k8s1.svg)
	<div style="text-align: right"> Source: https://kubernetes.io/docs/tutorials/kubernetes-basics/create-cluster/cluster-intro/ </div>

 - Each node runs `kubelet` that manages it and communicated with the master. It also runs Docker daemon (or other technologies) to manage containers.

 - The deploy sequence essentially involves the following:
   - `kubectl` tells the master to start application containers.
   - Master schedules containers to run.


#### Deployment

 - Once we have a cluster running, we can deploy containerized applications (one or many) using a *Deployment* configuration.
 - We can think of this as a set of instructions to K8s to set up the application on the cluster. Essentially, the containers are mapped to individual nodes.
 - After the mapping, a *Deployment Controller* keeps checking these instances allowing for **self-healing**.


![module_02_first_app](images/module_02_first_app.svg)
	<div style="text-align: right"> Source: https://kubernetes.io/docs/tutorials/kubernetes-basics/deploy-app/deploy-intro/ </div>

 - We create a deployment using a tool such as `kubectl` that uses an API to interact with the Master.
 - When creating a deployment, we need to specify container images as well as the number of copies of each image we want to instantiate.


#### Nodes and Pods

 - When an app is deployed as a container, it is encapsulated in a pod on a specific node.
 - A pod is a collection of containers that share a few things (see below) and are typically related to each other.
 	- Storage
 	- Networking and IP address
 - These related containers can be a web server and a database for instance.
 - We can also have a single container in a pod (this is what we will do here).
 - Pods are the most basic unit in K8s. It is pods that are created and destroyed, not individual containers.


<img src="images/module_03_pods.svg" alt="module_03_pods" style="width:800px;"/>
	<div style="text-align: right"> Source: https://kubernetes.io/docs/tutorials/kubernetes-basics/explore/explore-intro/ </div>


 - A node is essentially a VM/machine and can have multiple pods (depending on how K8s schedules pods). A node runs:
  - Kubelet: a program that communicates between the master and the node and manages the pods on the node.
  - Docker daemon (or equivalent) for pulling and running containers from images.


<img src="images/module_03_nodes.svg" alt="module_03_nodes" style="width:400px;"/>
	<div style="text-align: right"> Source: https://kubernetes.io/docs/tutorials/kubernetes-basics/explore/explore-intro/ </div>


#### Services

> A Kubernetes Service is an abstraction layer which defines a logical set of Pods and enables external traffic exposure, load balancing and service discovery for those Pods.
<div style="text-align: right"> Source: https://kubernetes.io/docs/tutorials/kubernetes-basics/expose/expose-intro/ </div>

 - It enables coupling between pods (each of which have a unique IP).
 - The pods with IPs cannot be typically accessed from the outside. Instead, a service can be used to allow external connections.
 - For example, a service with a *spec* that says `NodePort` exposed pods on the same port of each selected node in the cluster using NAT (Network Address Translation). We will see an example while deploying our model.


![module_04_services](images/module_04_services.svg)
<div style="text-align: right"> Source: https://kubernetes.io/docs/tutorials/kubernetes-basics/expose/expose-intro/ </div>

 - The Service abstraction lets pods get deleted and replicated in the cluster with no influence on our app.
 - A set of pods are matched to a service using labels and selectors. See [https://kubernetes.io/docs/tutorials/kubernetes-basics/expose/expose-intro/](https://kubernetes.io/docs/tutorials/kubernetes-basics/expose/expose-intro/) for more information.

#### Scaling and Updating

 - See [https://kubernetes.io/docs/tutorials/kubernetes-basics/scale/scale-intro/](https://kubernetes.io/docs/tutorials/kubernetes-basics/scale/scale-intro/) for how an app can scale.

 - See [https://kubernetes.io/docs/tutorials/kubernetes-basics/update/update-intro/](https://kubernetes.io/docs/tutorials/kubernetes-basics/update/update-intro/) for information on how to perform a *rolling update*.
#### Scaling and Updating

 - See [https://kubernetes.io/docs/tutorials/kubernetes-basics/scale/scale-intro/](https://kubernetes.io/docs/tutorials/kubernetes-basics/scale/scale-intro/) for how an app can scale.

 - See [https://kubernetes.io/docs/tutorials/kubernetes-basics/update/update-intro/](https://kubernetes.io/docs/tutorials/kubernetes-basics/update/update-intro/) for information on how to perform a *rolling update*.


### Installing kubectl, minikube, Virtualbox and Docker

 - We have already installed Docker for Desktop.


#### Virtualbox

 - Virtualbox is a software product for running virtual machines on a variety of host operating systems (MacOS/Windows/Linux).

 - Go to [https://www.virtualbox.org/](https://www.virtualbox.org/) and download the installer for your OS. The installation is straightforward.

 - Once installed, try to download a Linux distribution such as [Debian](https://www.debian.org/) or [Fedora](https://getfedora.org/) to try out how Virtualbox works.

![virtualbox](images/virtualbox.png)

#### Kubectl

 - `kubectl` is a client utility to talk to the the K8s server for container orchestration.

 - Download `kubectl` from [this page](https://kubernetes.io/docs/tasks/tools/install-kubectl/). Here is the example command for MacOS:


```bash
(datasci-dev) ttmac:k8s theja$ curl -LO "https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/darwin/amd64/kubectl"
```
 - Change its permissions to make it executable and move it to a place where it can be on the `$PATH`. Check whether you can access it by querying its version information.

```bash
(datasci-dev) ttmac:k8s theja$ ls -l
total 96696
-rw-r--r--  1 theja  staff  49458208 Sep 15 21:18 kubectl
(datasci-dev) ttmac:k8s theja$ chmod +x kubectl
(datasci-dev) ttmac:k8s theja$ mv kubectl ~/Library/local/bin/
(datasci-dev) ttmac:k8s theja$ kubectl version --client
Client Version: version.Info{Major:"1", Minor:"19", GitVersion:"v1.19.1", GitCommit:"206bcadf021e76c27513500ca24182692aabd17e", GitTreeState:"clean", BuildDate:"2020-09-09T11:26:42Z", GoVersion:"go1.15", Compiler:"gc", Platform:"darwin/amd64"}
```


#### Minikube

 - Minikube (`minikube`) is a tool that runs a single node K8s cluster in a virtual machine on our local computer. In particular, Minikube is a lightweight K8s implementation that creates a VM on your local machine and deploys a simple cluster containing only one node.

 - Follow the instructions for your operating system at [https://kubernetes.io/docs/tasks/tools/install-minikube/](https://kubernetes.io/docs/tasks/tools/install-minikube/). For instance, to install it on MacOS, we do the following.

 - We check if virtualization is supported. If we run the following command from the terminal, we expect the `VMX` acronym to be colored:

```bash
(datasci-dev) ttmac:k8s theja$ sysctl -a | grep -E --color 'machdep.cpu.features|VMX'
machdep.cpu.features: FPU VME DE PSE TSC MSR PAE MCE CX8 APIC SEP MTRR PGE MCA CMOV PAT PSE36 CLFSH DS ACPI MMX FXSR SSE SSE2 SS HTT TM PBE SSE3 PCLMULQDQ DTES64 MON DSCPL VMX EST TM2 SSSE3 CX16 TPR PDCM SSE4.1 SSE4.2 x2APIC POPCNT AES PCID XSAVE OSXSAVE TSCTMR AVX1.0 RDRAND F16C
```

 - We will download the stand-alone binary just as we did for `kubectl` and move it to the right path:

 ```bash
(datasci-dev) ttmac:k8s theja$ curl -Lo minikube https://storage.googleapis.com/minikube/releases/latest/minikube-darwin-amd64
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.5M  100 55.5M    0     0  10.9M      0  0:00:05  0:00:05 --:--:-- 11.7M
(datasci-dev) ttmac:k8s theja$ chmod +x minikube
(datasci-dev) ttmac:k8s theja$ mv minikube ~/Library/local/bin/
```

 - Check that its on the path and working correctly by using the following status check:

 ```bash
 (datasci-dev) ttmac:k8s theja$ minikube status
🤷  There is no local cluster named "minikube"
👉  To fix this, run: "minikube start"
(datasci-dev) ttmac:k8s theja$ minikube version
minikube version: v1.13.0
commit: 0c5e9de4ca6f9c55147ae7f90af97eff5befef5f-dirty
```
